# Code 4-15 to Code 4-27

In [ ]:
## This is written in Google Colab. Reccomended to be opened in Google Colab

In [ ]:
import pandas as pd
import nltk
pd.options.display.max_colwidth = 1000

In [ ]:
## This is Google colab upload

In [4]:
from google.colab import files
uploaded = files.upload()

Saving news_summary.csv to news_summary (1).csv


In [5]:
import io
import pandas as pd
t1 = pd.read_csv(io.BytesIO(uploaded['news_summary.csv']),encoding = 'latin1')

In [6]:

t1.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in offices order,http://www.hindustantimes.com/india-news/rakshabandhan-compulsory-in-daman-and-diu-women-employees-to-tie-rakhis-to-male-colleagues/story-E5h5U1ZDJii5zFpLXWRkhJ.html?utm_source=inshorts&utm_medium=referral&utm_campaign=fullarticle,The Administration of Union Territory Daman and Diu has revoked its order that made it compulsory for women to tie rakhis to their male colleagues on the occasion of Rakshabandhan on August 7. The administration was forced to withdraw the decision within 24 hours of issuing the circular after it received flak from employees and was slammed on social media.,"The Daman and Diu administration on Wednesday withdrew a circular that asked women staff to tie rakhis on male colleagues after the order triggered a backlash from employees and was ripped apart on social media.The union territory?s administration was forced to retreat within 24 hours of issuing the circular that made it compulsory for its staff to celebrate Rakshabandhan at workplace.?It has been decided to celebrate the festival of Rakshabandhan on August 7. In this connection, all offices/ departments shall remain open and celebrate the festival collectively at a suitable time wherein all the lady staff shall tie rakhis to their colleagues,? the order, issued on August 1 by Gurpreet Singh, deputy secretary (personnel), had said.To ensure that no one skipped office, an attendance report was to be sent to the government the next evening.The two notifications ? one mandating the celebration of Rakshabandhan (left) and the other withdrawing the mandate (right) ? were issued by the D..."
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorcing rich man',http://www.hindustantimes.com/bollywood/malaika-arora-khan-was-trolled-for-divorcing-a-rich-man-her-reply-is-dignity-itself/story-oIPZNI9deImCmabLMMwR2H.html?utm_source=inshorts&utm_medium=referral&utm_campaign=fullarticle,"Malaika Arora slammed an Instagram user who trolled her for ""divorcing a rich man"" and ""having fun with the alimony"". ""Her life now is all about wearing short clothes, going to gym or salon, enjoying vacation[s],"" the user commented. Malaika responded, ""You certainly got to get your damn facts right before spewing sh*t on me...when you know nothing about me.""","From her special numbers to TV?appearances, Bollywood actor Malaika Arora Khan has managed to carve her own identity. The actor, who made her debut in the Hindi film industry with the blockbuster debut opposite Shah Rukh Khan in Chaiyya Chaiyya from Dil Se (1998), is still remembered for the song. However, for trolls, she is a woman first and what matters right now is that she divorced a ?rich man?. On Wednesday, Malaika Arora shared a gorgeous picture of herself on Instagram and a follower decided to troll her for using her ?alumni? (read alimony) money to wear ?short clothes and going to gym or salon?. Little did he/she know that the Munni Badnam star would reply with the perfect comeback. Take a look at the interaction: Super excited to be affiliated with Khanna Jewellers @khannajewellerskj as their brand ambassador. Crafted to perfection, their stunning statement jewellery is a must have for every jewellery lover. #khannajewellers...#maksquad?? #hair @hairbypriyanka #styli..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS' form,http://www.hindustantimes.com/patna/bihar-igims-form-loses-virginity-after-row-opts-for-unmarried-instead/story-8XmHIg39otGBoZAYEBQxMK.html?utm_source=inshorts&utm_medium=referral&utm_campaign=fullarticle,"The Indira Gandhi Institute of Medical Sciences (IGIMS) in Patna on Thursday made corrections in its Marital Declaration Form by changing 'Virgin' option to 'Unmarried'. Earlier, Bihar Health Minister defined virgin as being an unmarried woman and did not consider the term obj

We convert our “headlines” column into a list of lists. It would be better to take the “ctext” as it contains a huge corpus of words. I have used the “headlines” column for an example 

In [7]:
t1["h1_stop"] = t1["headlines"].str.lower()

Lets stem the words befor we build word vectors.
Stemming is a process to bring back the words to their root forms and hence similar words could be normalized.  

In [8]:
from gensim.parsing.porter import PorterStemmer
p = PorterStemmer()
p.stem_documents(["He was dining", "We dined yesterday"])


['he wa dine', 'we dine yesterdai']

In [9]:
!pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32917 sha256=8c19aba134d118abf8e30184a480b5c268a0354cde17abca417f8ebfd3dbae38
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


Applying Stop Words and Stemming

In [10]:
import stop_words
eng_words = stop_words.get_stop_words('en')

for i in eng_words:
    if(len(i)>1):
        wrd_repl = r"\b" + i + r"\b"
        
        t1["h1_stop"] = t1["h1_stop"].str.replace(wrd_repl,"")
t1["h2_stop"] = pd.Series(p.stem_documents(t1["h1_stop"]))
    
sentences = list(t1["h2_stop"].str.split())
sentences[0:10]

[['daman',
  '&',
  'diu',
  'revok',
  'mandatori',
  'rakshabandhan',
  'offic',
  'order'],
 ['malaika', 'slam', 'user', 'troll', "'divorc", 'rich', "man'"],
 ["'virgin'", 'now', 'correct', "'unmarried'", "igims'", 'form'],
 ['aaj', 'aapn', 'pakad', 'liya:', 'let', 'man', 'dujana', 'kill'],
 ['hotel', 'staff', 'get', 'train', 'spot', 'sign', 'sex', 'traffick'],
 ['man',
  'found',
  'dead',
  'delhi',
  'polic',
  'station,',
  'kin',
  'alleg',
  'foul',
  'plai'],
 ['delhi', 'hc', 'reduc', 'aid', "'negligent'", 'accid', 'victim', '45%'],
 ['60-yr-old', 'lynch', 'rumour', 'cut', "people'", 'hair'],
 ['chopper', 'fly', 'critic', 'low', 'led', '2015', 'bombai', 'high', 'crash'],
 ['congress', 'open', "'state", 'bank', "tomato'", 'lucknow']]

We are now building the word2vec gensim model. The “size” option is the number of dimensions we want the word to be represented in. min_count is the minimum number of documents the word should be in. The “iter” option is the number of “epochs” of the neural network

In [12]:
import gensim
model = gensim.models.Word2Vec(iter=10,min_count=10,size=200) 
model.build_vocab(sentences) 
token_count = sum([len(sentence) for sentence in sentences])

In [13]:
model.train(sentences,total_examples = token_count,epochs = model.iter)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


(149396, 329250)

Set of word vectors generated

In [14]:
model.wv.vocab

{"'": <gensim.models.keyedvectors.Vocab at 0x7ff5e1f4a780>,
 "'s": <gensim.models.keyedvectors.Vocab at 0x7ff5e1f57b38>,
 ',': <gensim.models.keyedvectors.Vocab at 0x7ff5e1f514e0>,
 '1': <gensim.models.keyedvectors.Vocab at 0x7ff5e1f43390>,
 '10': <gensim.models.keyedvectors.Vocab at 0x7ff5e1f53518>,
 '100': <gensim.models.keyedvectors.Vocab at 0x7ff5e1ed97f0>,
 '12': <gensim.models.keyedvectors.Vocab at 0x7ff5e1edd2b0>,
 '15': <gensim.models.keyedvectors.Vocab at 0x7ff5e1f57940>,
 '16': <gensim.models.keyedvectors.Vocab at 0x7ff5e1ed9390>,
 '1st': <gensim.models.keyedvectors.Vocab at 0x7ff5e1eddcc0>,
 '2': <gensim.models.keyedvectors.Vocab at 0x7ff5e1edd2e8>,
 '20': <gensim.models.keyedvectors.Vocab at 0x7ff5e1f51b00>,
 '2016': <gensim.models.keyedvectors.Vocab at 0x7ff5e1ed9a58>,
 '2017': <gensim.models.keyedvectors.Vocab at 0x7ff5e1f431d0>,
 '3': <gensim.models.keyedvectors.Vocab at 0x7ff5e1f53390>,
 '30': <gensim.models.keyedvectors.Vocab at 0x7ff5e1edd400>,
 '4': <gensim.models.ke

In [15]:
model.most_similar('polic')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('report', 0.9998923540115356),
 ('get', 0.9998892545700073),
 ('take', 0.999885082244873),
 ('mumbai', 0.9998843669891357),
 ('indian', 0.9998838305473328),
 ('call', 0.9998794794082642),
 ('jail', 0.9998782277107239),
 ('year', 0.999877393245697),
 ('kill', 0.9998773336410522),
 ('delhi', 0.9998770952224731)]

Example of "Polic" word vector

In [16]:
model["polic"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.11834934,  0.21337458,  0.03328618, -0.21670192,  0.12195799,
        0.05638056,  0.3402187 , -0.18369518, -0.06047295,  0.05781244,
        0.04509117,  0.17188638, -0.26442727,  0.04853801, -0.25738654,
       -0.01403073,  0.2590166 ,  0.03056786, -0.13485454,  0.1301692 ,
        0.11417734,  0.1709013 ,  0.00435636, -0.02373919, -0.20038344,
       -0.18156202, -0.367141  ,  0.11897054,  0.0041351 ,  0.28917104,
       -0.04908011,  0.03572244,  0.10251857, -0.19918089,  0.03058901,
        0.16307989, -0.01174473, -0.04265049,  0.00520038, -0.01660416,
        0.2031985 , -0.02478904, -0.09122077, -0.02690028, -0.12100707,
        0.09115547,  0.0178129 , -0.01111273,  0.31348306,  0.01833618,
       -0.00068799, -0.02229044, -0.00497743,  0.09413265,  0.1540884 ,
       -0.25502607, -0.11291281, -0.06615365,  0.05772389, -0.0866123 ,
        0.03795764, -0.04062628,  0.11240582,  0.04045807,  0.07584371,
       -0.08662183, -0.01104501,  0.01471077, -0.11915521, -0.03

Out of Vocab word throws error

In [17]:
model['administration']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


KeyError: ignored

The other popular library for word2vec is fasttext. This is a library released by facebook. Primary difference is that the input vector being considered are set of “ngrams” of the word. For example “transform” would get in “tra”, “ran”, “ans” etc. This  helps the word2vec model to provide vector representation for most words as they will be broken down into ngrams and their weights recombined to give a final output. 

Preprocessing the sentences to get the required format

In [18]:
t1["h1_stop"] = t1["h1_stop"].str.replace('[0-9]+','numbrepl')
t1["h1_stop"] = t1["h1_stop"].str.replace('[^a-z\s]+','')
list2=[]
for i in t1["h1_stop"]:
  list1 = i.split()
  str1 = ""
  for j in list1:
    if(len(j)>=3):
      str1 = str1 + " " + j
  list2.append(str1)
t1["h1_proc"] = list2

In [20]:
t1[["headlines","h1_proc"]]

,headlines,h1_proc
0,Daman & Diu revokes mandatory Rakshabandhan in offices order,daman diu revokes mandatory rakshabandhan offices order
1,Malaika slams user who trolled her for 'divorcing rich man',malaika slams user trolled divorcing rich man
2,'Virgin' now corrected to 'Unmarried' in IGIMS' form,virgin now corrected unmarried igims form
3,Aaj aapne pakad liya: LeT man Dujana before being killed,aaj aapne pakad liya let man dujana killed
4,Hotel staff to get training to spot signs of sex trafficking,hotel staff get training spot signs sex trafficking
...,...,...
4509,Rasna seeking ?250 cr revenue from snack category: Chairman,rasna seeking numbrepl revenue snack category chairman
4510,Sachin attends Rajya Sabha after questions on attendance,sachin attends rajya sabha questions attendance
4511,Shouldn't rob their childhood: Aamir on kids reality shows,rob childhood aamir kids reality shows
4512,"Asha Bhosle gets ?53,000 power bill for unused bungalow",asha bhosle gets numbreplnumbrepl power bill unused bungalow


Have used word_punctuation_tokenizer to tokenize words. The punctuation tokenizer would treat the punctionations as separate tokens as compared to “tokenize” function from NLTK

In [21]:
word_punctuation_tokenizer = nltk.WordPunctTokenizer()

word_tokenized_corpus = [word_punctuation_tokenizer.tokenize(sent) 
  for sent in t1["h1_proc"]]


In [23]:
from gensim.models.fasttext import FastText as FT_gen
model_ft = FT_gen(word_tokenized_corpus,size=100,
                      window=5,
                      min_count=10,
                      
                      sg=1,
                      iter=100
                      )

Word vector for "police"

In [24]:
print(model_ft.wv['police'])

[-0.22546193 -0.57399255  0.37502384 -0.5541501  -0.3610318   1.2900186
 -0.21831332  0.44128725  0.5840549  -0.12127865  0.34372768  0.03986532
 -0.44284812 -0.2760494   0.54226714 -0.34788117 -0.15348753  1.0568762
  0.22936492 -0.4770464   0.11813045  0.05293164 -0.39436248 -0.00858558
  0.29899442  0.54440194  0.6654917   0.04176539 -0.0792782   0.3340478
 -0.48971924  0.50223035 -0.24313357  0.21165931  0.58307886  0.2431692
 -0.48484662 -0.03386339  0.41664338  0.5915182  -0.53992075  0.1489043
  0.6073883   0.5138187  -0.9025914  -0.14146869  0.45071426  0.30966145
 -0.11701576  0.24241701 -0.26873347 -0.28248525 -0.04691637 -0.05199096
  0.4091813   0.02940704  0.84898555  0.27491227 -0.05435127 -0.14587085
  0.87376994  0.35274458  0.4400529  -0.68847644 -0.31428352 -0.82805383
 -0.68499035 -0.42406824 -0.7662555   0.42705762  0.39762172 -0.17032546
 -0.04753472 -0.07993346  0.4266643   0.13033721  0.31655085 -0.1532629
 -0.11560907  0.16596934  0.2285682  -0.04258824  0.41395

In [25]:
model_ft.most_similar('police')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('injured', 0.4775485396385193),
 ('bomb', 0.4253961145877838),
 ('numbreplyrold', 0.3804662823677063),
 ('politics', 0.37814605236053467),
 ('notice', 0.37109649181365967),
 ('since', 0.3676730990409851),
 ('join', 0.3629407286643982),
 ('officers', 0.3592672049999237),
 ('quit', 0.3568107485771179),
 ('home', 0.35545021295547485)]

In [26]:
print(model_ft.wv.vocab)   # list of words in dictionary


{'mandatory': <gensim.models.keyedvectors.Vocab object at 0x7ff5ddb6afd0>, 'order': <gensim.models.keyedvectors.Vocab object at 0x7ff5ddb6ae10>, 'slams': <gensim.models.keyedvectors.Vocab object at 0x7ff5ddb6af60>, 'man': <gensim.models.keyedvectors.Vocab object at 0x7ff5ddb6a940>, 'now': <gensim.models.keyedvectors.Vocab object at 0x7ff5ddb6aa20>, 'let': <gensim.models.keyedvectors.Vocab object at 0x7ff5ddb6aa58>, 'killed': <gensim.models.keyedvectors.Vocab object at 0x7ff5ddb6add8>, 'staff': <gensim.models.keyedvectors.Vocab object at 0x7ff5ddb6ac88>, 'get': <gensim.models.keyedvectors.Vocab object at 0x7ff5ddb6a9e8>, 'sex': <gensim.models.keyedvectors.Vocab object at 0x7ff5ddb6a978>, 'found': <gensim.models.keyedvectors.Vocab object at 0x7ff5ddb6a8d0>, 'dead': <gensim.models.keyedvectors.Vocab object at 0x7ff5ddb6a780>, 'delhi': <gensim.models.keyedvectors.Vocab object at 0x7ff5ddb6a748>, 'police': <gensim.models.keyedvectors.Vocab object at 0x7ff5ddb6abe0>, 'station': <gensim.model

We test an out of vocabulary word. Unlike earlier there is a word vector provided because of "fast text" n-grams logic 

In [27]:
print ('administration' in model_ft.wv.vocab)
model_ft["administration"]

False


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


array([-4.1486531e-01, -2.5789216e-01,  3.4350243e-01, -2.3928504e-01,
        1.4261907e-01,  5.6011325e-01, -2.4807902e-02, -1.8245694e-01,
        4.1095342e-02,  4.2312241e-01,  5.0690796e-02,  6.7387927e-01,
       -4.3783885e-01,  3.4516763e-02,  8.4873132e-02,  2.7353689e-01,
       -1.0073330e+00, -1.4384724e-01,  6.2248522e-01, -2.8178346e-01,
       -1.7427523e-01, -6.2907404e-01,  3.0742498e-02,  1.4868927e-01,
       -5.2438319e-01,  6.1059934e-01,  1.3458890e-01, -5.3849173e-01,
        1.1654196e-01, -1.8144158e-01, -4.6565360e-01,  3.4537533e-01,
       -4.5373538e-01, -5.4092699e-01,  4.6502265e-01,  2.5856513e-01,
       -4.5956773e-01,  2.5455939e-02,  2.1746261e-02,  1.6322225e-01,
        1.2891193e-02,  4.5961207e-01, -3.6925638e-01, -6.2095106e-01,
       -2.8730270e-01,  4.6954539e-01,  5.2485406e-01, -1.6333321e-01,
       -3.8108230e-01,  5.9174895e-01,  2.3605470e-01,  6.9409323e-01,
       -9.8086916e-02,  7.6792854e-01,  3.3083907e-01,  5.2006286e-01,
      

Finding most similar for out of vocabulary word

In [28]:
model_ft.most_similar('administration')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('ministry', 0.8346989154815674),
 ('ministers', 0.6878262162208557),
 ('minister', 0.6578146815299988),
 ('resignation', 0.6402623653411865),
 ('min', 0.631399393081665),
 ('station', 0.6125809550285339),
 ('politics', 0.605400800704956),
 ('stations', 0.5941411256790161),
 ('union', 0.5542712211608887),
 ('railway', 0.5237174034118652)]